In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
import os.path
import os,re
import glob
import pprint
import pathlib
import threading
import time
import joblib
from pandas import DataFrame
from datetime import datetime,timedelta
import datetime
from sklearn.utils import shuffle

In [11]:
# Korean
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/Hancom Gothic Regular.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rcParams['axes.unicode_minus'] = False

In [2]:
plt.rcParams['axes.unicode_minus'] = False
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina'")

### Raw Data

In [3]:
def making_data(Kw,Type,num):
    dir_path = "C:/Users/SM-PC/BDJ/기계시설물 고장 예지 센서/Training/vibration/vibration"
    files = glob.glob(os.path.join(dir_path+'/'+Kw+'/*/'+Type,'*.*'),
                      recursive=True)[:num]
    pieces = []
    files_arr = []
    for i in range(len(files)):
        file = files[i]       
        frame = pd.read_csv(file,skiprows=8)
        frame.drop(frame.columns[[2]],axis=1,inplace=True)
        frame.columns=['time','data']
        frame.set_index('time',inplace=True)
        pieces.append(frame)
    df = pd.concat(pieces,ignore_index=True,axis=1)
    return df

### Kind of states and power

#### 1. Kind of states
- #####  정상 : Normal, 
- ##### 베어링 : Bearing defect, 
- ##### 회전체: Rotational defect, 
- ##### 축정렬: Axis missalignment, 
- ##### 벨트: Belt looseness

#### 2. Kind of power with states 
- 2.2 :  Normal, Bearing,Rotation,Axis,Belt 
- 3.7 : Normal, Bearing,Rotation,Axis 
- 3.75 : Normal, Axis
- 5.5 : Normal, Bearing, Rotation,Axis,Belt
- 7.5 :  Normal, Bearing,Axis,Belt
- 11 : Normal, Bearing,Rotation,Axis,Belt 
- 15 : Normal, Bearing,Rotation,Belt 
- 18.5 : Normal, Bearing,Belt 
- 22 : Normal,Rotation,Axis,Belt 
- 30 : Normal, Axis
- 37 : Normal, Axis 
- 55 : Normal, Rotation,Belt 

### Making DataFrame per each state with power

In [ ]:
# normal
kw = ['2.2kW', '3.7kW', '3.75kW', '5.5kW','7.5kW','11kW', '15kW', '18.5kW', '22kW', '30kW', '37kW', '55kW']
arr = []
for i in range(len(kw)):
    a = making_data(kw[i],'정상',10000)
    arr.append(a)
    a_name = ['2d2', '3d7', '3d75', '5d5','7d5','11', '15', '18d5', '22', '30', '37', '55']
    globals()['normal{}'.format(a_name[i])]=arr[i]

In [90]:
# bearing
kw = ['2.2kW', '3.7kW', '5.5kW','7.5kW','11kW', '15kW', '18.5kW']
arr = []
for i in range(len(kw)):
    a = making_data(kw[i],'베어링불량',1000)
    arr.append(a)
    a_name = ['2d2', '3d7', '5d5','7d5','11', '15', '18d5']
    globals()['bearing{}'.format(a_name[i])]=arr[i]

In [91]:
# rotating
kw = ['2.2kW', '3.7kW', '5.5kW','11kW', '15kW', '22kW', '55kW']
arr = []
for i in range(len(kw)):
    a = making_data(kw[i],'회전체불평형',1000)
    arr.append(a)
    a_name = ['2d2', '3d7', '5d5', '11', '15', '22', '55']  
    globals()['rotating{}'.format(a_name[i])]=arr[i]

In [93]:
# axis
kw = ['2.2kW', '3.7kW', '3.75kW', '5.5kW','7.5kW','11kW', '22kW', '30kW', '37kW']
arr = []
for i in range(len(kw)):
    a = making_data(kw[i],'축정렬불량',1000)
    arr.append(a)
    a_name = ['2d2', '3d7', '3d75', '5d5','7d5','11', '22', '30', '37']
    globals()['axis{}'.format(a_name[i])]=arr[i]

In [95]:
# belt looseness
kw = ['2.2kW', '5.5kW','7.5kW','11kW', '15kW', '18.5kW', '22kW', '55kW']
for i in range(len(kw)):
    a = making_data(kw[i],'벨트느슨함',1000)
    arr.append(a)
    a_name = ['2d2', '5d5','7d5','11', '15', '18d5', '22', '55']
    globals()['belt{}'.format(a_name[i])]=arr[i]

### Making Spectrogram

In [96]:
import pylab 
from scipy import signal
path = r'C:/Users/SM-PC/BDJ/BDJ_Spectrogram_deep'

In [97]:
import time
import threading

In [110]:
def spectrogram1(df):
    ## Spectrogram type 1
    Fs=4000
    f, tt, Sxx = signal.spectrogram(df, Fs)
    plt.pcolormesh(tt, f, Sxx, shading='gouraud')
    plt.ylim(0, 2000)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(path+'/sp1',Type+'_'+kw[j]+'_'+str(i)+'.jpg'))
    plt.close()

In [111]:
kw = ['2d2', '3d7', '3d75', '5d5','7d5','11', '15', '18d5', '22', '30', '37', '55']
Type = 'normal'

In [112]:
#change kw and Type
start  = time.time()
for i in range(1000):
    for j in range(len(kw)):
            spectrogram1(globals()['normal{}'.format(kw[j])][i])

end = time.time()
sec = (end - start)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])

0:13:24


In [113]:
kw = ['2d2', '3d7', '5d5','7d5','11', '15', '18d5']
Type = 'bearing'

In [114]:
#change kw and Type
start  = time.time()
for i in range(1000):
    for j in range(len(kw)):
            spectrogram1(globals()['bearing{}'.format(kw[j])][i])

end = time.time()
sec = (end - start)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])

0:08:30


In [115]:
kw = ['2d2', '3d7', '5d5', '11', '15', '22', '55']
Type = 'rotating'

In [116]:
#change kw and Type
start  = time.time()
for i in range(1000):
    for j in range(len(kw)):
            spectrogram1(globals()['rotating{}'.format(kw[j])][i])

end = time.time()
sec = (end - start)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])

0:08:13


In [117]:
kw = ['2d2', '3d7', '3d75', '5d5','7d5','11', '22', '30', '37']
Type = 'axis'

In [118]:
#change kw and Type
start  = time.time()
for i in range(1000):
    for j in range(len(kw)):
            spectrogram1(globals()['axis{}'.format(kw[j])][i])

end = time.time()
sec = (end - start)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])

0:10:35


In [119]:
kw = ['2d2', '5d5','7d5','11', '15', '18d5', '22', '55']
Type = 'belt'

In [120]:
#change kw and Type
start  = time.time()
for i in range(1000):
    for j in range(len(kw)):
            spectrogram1(globals()['belt{}'.format(kw[j])][i])

end = time.time()
sec = (end - start)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])

0:09:25
